In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from  google.colab import files
uploaded = files.upload()

Saving email.csv to email.csv


In [ ]:
df= pd.read_csv('email.csv')

In [ ]:
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
missing_values = df.isnull().sum()
print(missing_values)

Category    0
Message     0
dtype: int64


### Considering true news as label 0 and fake news as label 1




In [ ]:
# Replace values using replace() method
df['Category'] = df['Category'].replace({'ham': 0, 'spam': 1})

# Display the modified DataFrame
print(df)

            Category                                            Message
0                  0  Go until jurong point, crazy.. Available only ...
1                  0                      Ok lar... Joking wif u oni...
2                  1  Free entry in 2 a wkly comp to win FA Cup fina...
3                  0  U dun say so early hor... U c already then say...
4                  0  Nah I don't think he goes to usf, he lives aro...
...              ...                                                ...
5568               0               Will ü b going to esplanade fr home?
5569               0  Pity, * was in mood for that. So...any other s...
5570               0  The guy did some bitching but I acted like i'd...
5571               0                         Rofl. Its true to its name
5572  {"mode":"full"                                    isActive:false}

[5573 rows x 2 columns]


In [ ]:
df.head(5)

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.shape

(5573, 2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5573 entries, 0 to 5572
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5573 non-null   object
 1   Message   5573 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


## Text Cleaning

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps= PorterStemmer()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
corpus=[]
for i in range(len(df)):
    rp= re.sub('[^ a-zA-Z]',"",df['Message'][i])
    rp= rp.lower()
    rp= rp.split()
    rp= (ps.stem(word) for word in rp if not word in set(stopwords.words('english')))
    rp= "".join(rp)
    corpus.append(rp)

### Vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
X= cv.fit_transform(corpus).toarray()

In [ ]:
y=df['Category']

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Convert labels to string type before encoding
y_str = y.astype(str)  # Ensure all labels are strings

# Convert labels to numeric format
y_numeric = label_encoder.fit_transform(y_str)  # Use the string version of labels

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y_numeric, test_size=0.2, random_state= 0)

In [ ]:
print(y_train.dtype)

print(type(y_train))

int64
<class 'numpy.ndarray'>


In [ ]:
print(y_train.shape)

(4458,)


### NAive-bais

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model= MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [ ]:
ytest_pred= model.predict(X_test)
ypred_train= model.predict(X_train)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print('Train Accuracy:', accuracy_score(y_train,ypred_train))
print('Test Accuracy:', accuracy_score(y_test,ytest_pred))

Train Accuracy: 0.8676536563481382
Test Accuracy: 0.8672645739910314


#logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression()
model_lr.fit(X_train,y_train)

LogisticRegression()

In [ ]:
# Predicting the model
y_predict_lr = model_lr.predict(X_test)

In [ ]:
# Finding accuracy, precision, recall and confusion matrix
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_predict_lr))


0.874439461883408


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Calculate the metrics
accuracy = accuracy_score(y_test, y_predict_lr)
precision = precision_score(y_test, y_predict_lr)
recall = recall_score(y_test, y_predict_lr)
conf_matrix = confusion_matrix(y_test, y_predict_lr)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.874439461883408
Precision: 1.0
Recall: 0.06040268456375839
Confusion Matrix:
 [[966   0]
 [140   9]]


#SVM

In [ ]:
# Training the model
from sklearn.svm import SVC
rbf_svc = SVC(kernel='rbf', C=100, random_state=10).fit(X_train,y_train)

In [ ]:
# Predicting the model
y_predict_svm = rbf_svc.predict(X_test)

In [ ]:
# Finding accuracy, precision, recall and confusion matrix
print(accuracy_score(y_test,y_predict_svm))


0.9085201793721973


#random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [ ]:
# Initialize Random Forest model
model_rf = RandomForestClassifier()

# Fit the model to the training data
model_rf.fit(X_train, y_train)

# Make predictions on the test data
y_predict_rf = model_rf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_predict_rf)
precision = precision_score(y_test, y_predict_rf)
recall = recall_score(y_test, y_predict_rf)
conf_matrix = confusion_matrix(y_test, y_predict_rf)

# Print the results
print("Random Forest Results:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("Confusion Matrix:\n", conf_matrix)

Random Forest Results:
Accuracy: 0.9085201793721973
Precision: 1.0
Recall: 0.31543624161073824
Confusion Matrix:
 [[966   0]
 [102  47]]
